In [35]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
from pinecone import Pinecone, ServerlessSpec
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [39]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
   name="aiprofessor", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region='us-east-1')
)

In [37]:
import json
data = json.load(open('reviews.json'))

In [30]:
processed_data = []
client = HuggingFaceEmbeddings()

for review in data['reviews']:
   embedding = client.embed_query(review['review'])

   processed_data.append({
      "values": embedding,
      "id": review["professor"],
      "metadata": {
         "review": review["review"],
         "department": review["department"],
         "teaching_quality": review["teaching_quality"],
         "workload": review["workload"],
         "difficulty": review["difficulty"],
         "overall_rating": review["overall_rating"],
      }
   })

   

/opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
processed_data[0]

In [40]:
index = pc.Index('aiprofessor')

index.upsert(
   vectors=processed_data,
   namespace='ns1'
)



{'upserted_count': 30}

In [41]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30}